# Logistics Shortest Path Demo
This notebook demonstrates the shortest path finding functionality using interactive maps.

In [7]:
# Import required libraries
import folium
from ipyleaflet import Map, Marker, Polyline
from geopy.distance import geodesic
import networkx as nx
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

# Import our custom route optimizer
from route_optimizer import RouteOptimizer

In [8]:
class InteractiveMapOptimizer:
    def __init__(self):
        self.optimizer = RouteOptimizer()
        self.points = []
        self.setup_ui()
    
    def setup_ui(self):
        # Create input fields
        self.name_input = widgets.Text(description='Location Name:', value='Point 1')
        self.lat_input = widgets.FloatText(description='Latitude:', value=17.3850)
        self.lon_input = widgets.FloatText(description='Longitude:', value=78.4867)
        
        # Create buttons
        self.add_button = widgets.Button(description='Add Point', button_style='primary')
        self.find_path_button = widgets.Button(description='Find Path', button_style='success')
        
        # Create dropdowns for start and end points
        self.start_point = widgets.Dropdown(description='Start:', options=[])
        self.end_point = widgets.Dropdown(description='End:', options=[])
        
        # Create map
        self.map = Map(center=(17.3850, 78.4867), zoom=10)
        
        # Create output area
        self.output = widgets.Output()
        
        # Set up event handlers
        self.add_button.on_click(self.add_point)
        self.find_path_button.on_click(self.find_path)
        
        # Create layout
        inputs = widgets.VBox([
            self.name_input,
            self.lat_input,
            self.lon_input,
            widgets.HBox([self.add_button]),
            widgets.HBox([self.start_point, self.end_point]),
            self.find_path_button
        ])
        
        # Display everything
        display(inputs)
        display(self.map)
        display(self.output)
    
    def add_point(self, b):
        name = self.name_input.value
        lat = self.lat_input.value
        lon = self.lon_input.value
        
        # Add to optimizer
        self.optimizer.add_location(name, lat, lon)
        self.points.append(name)
        
        # Add marker to map
        marker = Marker(location=(lat, lon), draggable=False, title=name)
        self.map.add_layer(marker)
        
        # Update dropdowns
        self.start_point.options = self.points
        self.end_point.options = self.points
        
        # Update next point name
        self.name_input.value = f'Point {len(self.points) + 1}'
        
        with self.output:
            clear_output()
            print(f'Added {name} at ({lat}, {lon})')
    
    def find_path(self, b):
        start = self.start_point.value
        end = self.end_point.value
        
        if not start or not end:
            with self.output:
                clear_output()
                print('Please select both start and end points')
            return
        
        # Build distance matrix
        self.optimizer.build_distance_matrix()
        
        # Find shortest path
        path, distance = self.optimizer.find_shortest_path(start, end)
        
        if path:
            # Create path coordinates
            path_coords = [(self.optimizer.locations[p][0], self.optimizer.locations[p][1]) for p in path]
            
            # Draw path on map
            line = Polyline(
                locations=path_coords,
                color='red',
                fill=False
            )
            self.map.add_layer(line)
            
            with self.output:
                clear_output()
                print(f'Shortest path: {" -> ".join(path)}')
                print(f'Total distance: {distance:.2f} km')
        else:
            with self.output:
                clear_output()
                print('No path found between selected points')

In [9]:
# Create and display the interactive map optimizer
optimizer = InteractiveMapOptimizer()

Map(center=[17.385, 78.4867], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Output()